In [1]:
import os

os.chdir('..')

import numpy as np
import json

import plotly.io as pio

from apps.data import *

pio.templates.default = "plotly_dark"
pio.renderers.default = "browser"

In [2]:
UNDER_OVER = 56.064

In [3]:
with open('C:/Users/jrnas/Downloads/BBGM_League_1_2220_free_agency.json', encoding='latin') as f:
    r_json = json.load(f)

In [4]:
df = player_json_to_df(r_json)

In [5]:
df['vorp_norm'] = (df['vorp'] / df['min']) * (32 * 82)

In [6]:
test_df = df[(df['min'] > 10) & (~df['vorp_norm'].isna())].reset_index(drop=True)
test_df['vorp_norm_wt'] = test_df['vorp_norm'] * test_df['min']
agg_df = test_df.groupby('ovr')[['vorp_norm_wt', 'min']].sum().reset_index()
agg_df['vorp_norm'] = agg_df['vorp_norm_wt'] / agg_df['min']

In [7]:
model_df_under = agg_df[agg_df['ovr'] <= UNDER_OVER].reset_index(drop=True)
poly_under = np.polyfit(model_df_under['ovr'], model_df_under['vorp_norm'], 1)

model_df_over = agg_df[agg_df['ovr'] > UNDER_OVER].reset_index(drop=True)
poly_over = np.polyfit(model_df_over['ovr'], model_df_over['vorp_norm'], 1)

df['vorp_under'] = np.polyval(poly_under, df['ovr'])
df['vorp_over'] = np.polyval(poly_over, df['ovr'])
df['vorp_pred'] = np.where(
    df['ovr'] <= UNDER_OVER,
    df['vorp_under'],
    df['vorp_over']
)

np.save('models/poly_under.npy', poly_under)
np.save('models/poly_over.npy', poly_over)

In [ ]:
df[['']]

In [26]:
df['cvorp'] = df['vorp'].clip(0, )
df['vorp_pct'] = df['vorp_pred'].clip(0, ) / df.groupby('season').cvorp.sum().mean()
df['vorp_pct_cap'] = df['vorp_pct'] * 30

In [29]:
df[df.lastName == 'Wembanyama'][['season', 'ovr', 'vorp', 'vorp_pred', 'vorp_pct', 'vorp_pct_cap']]

,season,ovr,vorp,vorp_pred,vorp_pct,vorp_pct_cap
218,2024,62,2.181748,3.387400,0.007813,0.234381
219,2025,64,2.961489,4.275119,0.009860,0.295804
220,2026,68,5.27007,6.050555,0.013955,0.418650
221,2027,66,4.283094,5.162837,0.011908,0.357227
222,2028,66,3.744058,5.162837,0.011908,0.357227
223,2029,64,3.915604,4.275119,0.009860,0.295804
224,2030,66,3.503258,5.162837,0.011908,0.357227
225,2031,64,3.632496,4.275119,0.009860,0.295804
226,2032,61,2.85734,2.943541,0.006789,0.203669
227,2033,60,2.083465,2.499682,0.005765,0.172958
